table of contents
1. Prepare
2. Data Import, Encoding
3. EDA
4. write df

In [1]:
import os, random, time

import numpy as np
import pandas as pd

# Prepare

## Define parameters

In [2]:
base_dir = os.path.join('..','..')
data_dir = os.path.join(base_dir, 'data')
raw_dir = os.path.join(data_dir, 'raw')
processed_dir = os.path.join(data_dir, 'processed')

infile_list = os.listdir(processed_dir)
for i, infile in enumerate(infile_list):
    print(i, infile)

0 events.csv
1 matches
2 events
3 passing.csv
4 coaches.csv
5 .~lock.teams.csv#
6 passing.zip
7 passing_match.csv
8 teams.csv


## Data import

In [3]:
competition = 'Spain'
#df = pd.read_csv(os.path.join(processed_dir+'/events', 'events_'+competition+'.csv'))
df = pd.read_json(os.path.join(raw_dir+'/events', 'events_'+competition+'.json'))
teams_df = pd.read_json(os.path.join(raw_dir, 'teams.json'))
players_df = pd.read_json(os.path.join(raw_dir, 'players.json'))
matches_df = pd.read_json(os.path.join(raw_dir+'/matches', 'matches_'+competition+'.json'))

In [4]:
#players_df['shortName'] = players_df.shortName.apply(lambda x: x.encode('utf-8').decode('unicode-escape'))
#teams_df['officialName'] = teams_df.officialName.apply(lambda x: x.encode('utf-8').decode('unicode-escape'))
#teams_df['name'] = teams_df.name.apply(lambda x: x.encode('utf-8').decode('unicode-escape'))


In [5]:
df.head()

,eventId,subEventName,tags,playerId,positions,matchId,eventName,teamId,matchPeriod,eventSec,subEventId,id
0,8,Simple pass,[{'id': 1801}],3542,"[{'y': 61, 'x': 37}, {'y': 50, 'x': 50}]",2565548,Pass,682,1H,2.994582,85,180864419
1,8,Simple pass,[{'id': 1801}],274435,"[{'y': 50, 'x': 50}, {'y': 30, 'x': 45}]",2565548,Pass,682,1H,3.137020,85,180864418
2,8,Simple pass,[{'id': 1801}],364860,"[{'y': 30, 'x': 45}, {'y': 12, 'x': 38}]",2565548,Pass,682,1H,6.709668,85,180864420
3,8,Simple pass,[{'id': 1801}],3534,"[{'y': 12, 'x': 38}, {'y': 69, 'x': 32}]",2565548,Pass,682,1H,8.805497,85,180864421
4,8,Simple pass,[{'id': 1801}],3695,"[{'y': 69, 'x': 32}, {'y': 37, 'x': 31}]",2565548,Pass,682,1H,14.047492,85,180864422


In [6]:
df[['st_x', 'st_y', 'ed_x', 'ed_y']] = df.apply(lambda xs: [xs['positions'][0]['x'], xs['positions'][0]['y'], xs['positions'][1]['x'], xs['positions'][1]['y']] if (len(xs['positions']) >= 2) else [xs['positions'][0]['x'], xs['positions'][0]['y'],None, None] , result_type='expand', axis=1)

In [7]:
teams_df_indexed = teams_df.set_index('wyId')

df = pd.merge(df, teams_df_indexed, left_on='teamId', right_index=True)

In [8]:
players_df_indexed = players_df.set_index('wyId')

df = pd.merge(df, players_df_indexed, left_on='playerId', right_index=True)

In [9]:
matches_df_indexed = matches_df.set_index('wyId')

df = pd.merge(df, matches_df_indexed, left_on='matchId', right_index=True)

In [41]:
#df.loc[df.eventName=='Shot'].head(30)
df.iloc[1890]

eventId                                                                  8
subEventName                                                   Simple pass
tags                                                        [{'id': 1801}]
playerId                                                             25472
positions                           [{'y': 3, 'x': 82}, {'y': 6, 'x': 67}]
matchId                                                            2565558
eventName                                                             Pass
teamId                                                                 682
matchPeriod                                                             1H
eventSec                                                            1618.6
subEventId                                                              85
id                                                               182021285
st_x                                                                    82
st_y                     

In [11]:
df.st_x.dtype

dtype('float64')

In [12]:
df_pass = df.loc[df.eventName=='Pass'].copy()

In [43]:
df_pass.head()
df_pass.iloc[243]

eventId                                                                  8
subEventName                                                   Simple pass
tags                                                        [{'id': 1801}]
playerId                                                              3277
positions                         [{'y': 39, 'x': 38}, {'y': 70, 'x': 35}]
matchId                                                            2565548
eventName                                                             Pass
teamId                                                                 682
matchPeriod                                                             1H
eventSec                                                           549.464
subEventId                                                              85
id                                                               180864501
st_x                                                                    38
st_y                     

In [14]:
#df_pass['st_x'] = None
#df_pass['st_y'] = None
#df_pass['ed_x'] = None
#df_pass['ed_y'] = None

#df_pass['list_positions'] = df_pass['positions'].apply(lambda x: list(x))

#df_pass['list_positions'].values
#df_pass[['st_x', 'st_y', 'ed_x', 'ed_y']] = df_pass.apply(lambda xs: [xs['positions'][0]['x'], xs['positions'][0]['y'], xs['positions'][1]['x'], xs['positions'][1]['y']], result_type='expand', axis=1)
#df_pass['st_x'] = df_pass.apply(lambda xs: xs['positions'][0]['x'])


df_pass['success'] = df_pass.apply(lambda xs: 1 if 1801 in [tag['id'] for tag in xs['tags']] else 0, axis=1)

In [15]:
df_pass.head(20)

,eventId,subEventName,tags,playerId,positions,matchId,eventName,teamId,matchPeriod,eventSec,...,seasonId,dateutc,winner,venue,label,date,referees,duration,competitionId,success
0,8,Simple pass,[{'id': 1801}],3542,"[{'y': 61, 'x': 37}, {'y': 50, 'x': 50}]",2565548,Pass,682,1H,2.994582,...,181144,2017-08-21 18:15:00,695,Estadio Ciudad de Valencia,"Levante - Villarreal, 1 - 0","August 21, 2017 at 8:15:00 PM GMT+2","[{'refereeId': 395056, 'role': 'referee'}, {'r...",Regular,795,1
39,8,Simple pass,[{'id': 1801}],3542,"[{'y': 48, 'x': 39}, {'y': 65, 'x': 26}]",2565548,Pass,682,1H,166.859710,...,181144,2017-08-21 18:15:00,695,Estadio Ciudad de Valencia,"Levante - Villarreal, 1 - 0","August 21, 2017 at 8:15:00 PM GMT+2","[{'refereeId': 395056, 'role': 'referee'}, {'r...",Regular,795,1
43,8,High pass,"[{'id': 901}, {'id': 1802}]",3542,"[{'y': 67, 'x': 32}, {'y': 42, 'x': 74}]",2565548,Pass,682,1H,181.665639,...,181144,2017-08-21 18:15:00,695,Estadio Ciudad de Valencia,"Levante - Villarreal, 1 - 0","August 21, 2017 at 8:15:00 PM GMT+2","[{'refereeId': 395056, 'role': 'referee'}, {'r...",Regular,795,0
50,8,Simple pass,[{'id': 1801}],3542,"[{'y': 60, 'x': 35}, {'y': 97, 'x': 42}]",2565548,Pass,682,1H,213.736626,...,181144,2017-08-21 18:15:00,695,Estadio Ciudad de Valencia,"Levante - Villarreal, 1 - 0","August 21, 2017 at 8:15:00 PM GMT+2","[{'refereeId': 395056, 'role': 'referee'}, {'r...",Regular,795,1
60,8,Head pass,[{'id': 1801}],3542,"[{'y': 25, 'x': 26}, {'y': 9, 'x': 27}]",2565548,Pass,682,1H,278.645906,...,181144,2017-08-21 18:15:00,695,Estadio Ciudad de Valencia,"Levante - Villarreal, 1 - 0","August 21, 2017 at 8:15:00 PM GMT+2","[{'refereeId': 395056, 'role': 'referee'}, {'r...",Regular,795,1
85,8,Smart pass,"[{'id': 901}, {'id': 1802}]",3542,"[{'y': 61, 'x': 38}, {'y': 47, 'x': 54}]",2565548,Pass,682,1H,354.921472,...,181144,2017-08-21 18:15:00,695,Estadio Ciudad de Valencia,"Levante - Villarreal, 1 - 0","August 21, 2017 at 8:15:00 PM GMT+2","[{'refereeId': 395056, 'role': 'referee'}, {'r...",Regular,795,0
105,8,Simple pass,[{'id': 1801}],3542,"[{'y': 71, 'x': 32}, {'y': 96, 'x': 42}]",2565548,Pass,682,1H,440.315069,...,181144,2017-08-21 18:15:00,695,Estadio Ciudad de Valencia,"Levante - Villarreal, 1 - 0","August 21, 2017 at 8:15:00 PM GMT+2","[{'refereeId': 395056, 'role': 'referee'}, {'r...",Regular,795,1
199,8,Simple pass,[{'id': 1801}],3542,"[{'y': 77, 'x': 45}, {'y': 72, 'x': 32}]",2565548,Pass,682,1H,844.316154,...,181144,2017-08-21 18:15:00,695,Estadio Ciudad de Valencia,"Levante - Villarreal, 1 - 0","August 21, 2017 at 8:15:00 PM GMT+2","[{'refereeId': 395056, 'role': 'referee'}, {'r...",Regular,795,1
205,8,Simple pass,[{'id': 1801}],3542,"[{'y': 87, 'x': 42}, {'y': 38, 'x': 38}]",2565548,Pass,682,1H,862.919269,...,181144,2017-08-21 18:15:00,695,Estadio Ciudad de Valencia,"Levante - Villarreal, 1 - 0","August 21, 2017 at 8:15:00 PM GMT+2","[{'refereeId': 395056, 'role': 'referee'}, {'r...",Regular,795,1
240,8,Simple pass,[{'id': 1801}],3542,"[{'y': 23, 'x': 30}, {'y': 19, 'x': 30}]",2565548,Pass,682,1H,1004.295944,...,181144,2017-08-21 18:15:00,695,Estadio Ciudad de Valencia,"Levante - Villarreal, 1 - 0","August 21, 2017 at 8:15:00 PM GMT+2","[{'refereeId': 395056, 'role': 'referee'}, {'r...",Regular,795,1


## Match Result

In [37]:
df_pass[['won','drawn', 'lost']] = df_pass.apply(lambda x: [1 if x['winner']==x['teamId'] else 0, 1 if x['winner']==0 else 0, 1 if (x['winner']!=x['teamId'] and x['winner']!=0) else 0], result_type='expand', axis=1)

### Calculate sectors of the playing field

Divide the land into 18 rectangles, we have a grid of 6 x 3 that mark 18 zones (https://footballbh.net/2019/02/04/the-use-of-zone-14-the-key-to-winning-football-matches/)

Zones

* Horizontal zone 
    * left 
    * center 
    * right
* Vertical zone  
    * own field (low, middle, high) 
    * rival field (low, middle, high)


In [16]:
#sector           = ('own', 'rival')
vertical_zones   = (('left',(0,25)),('middle', (25,75)),('right', (75,100)))
horizontal_zones = (('own_low', (0, 15)), ('own_middle', (15, 25)), ('own_high', (25, 50)),('rival_low', (50, 75)), ('rival_middle', (75, 85)), ('rival_high', (85, 100)))


#df_pass[list(map(lambda v : 'v_'+ v[0], vertical_zones))] = df_pass.apply(lambda x: list(map(lambda v : v[1], vertical_zones)), result_type='expand', axis=1)


In [17]:
df_pass[list(map(lambda v : 'v_'+ v[0], vertical_zones))] = df_pass.apply(lambda x: list(map(lambda v : 1 if (v[1][0] < x['st_x'] <= v[1][1]) else 0, vertical_zones)), result_type='expand', axis=1)

In [18]:
df_pass[list(map(lambda v : 'h_'+ v[0], horizontal_zones))] = df_pass.apply(lambda x: list(map(lambda v : 1 if (v[1][0] < x['st_y'] <= v[1][1]) else 0, horizontal_zones)), result_type='expand', axis=1)

In [40]:
#df_pass.reindex()
df_pass.head(30)


,eventId,subEventName,tags,playerId,positions,matchId,eventName,teamId,matchPeriod,eventSec,...,v_right,h_own_low,h_own_middle,h_own_high,h_rival_low,h_rival_middle,h_rival_high,won,drawn,lost
0,8,Simple pass,[{'id': 1801}],3542,"[{'y': 61, 'x': 37}, {'y': 50, 'x': 50}]",2565548,Pass,682,1H,2.994582,...,0,0,0,0,1,0,0,0,0,1
39,8,Simple pass,[{'id': 1801}],3542,"[{'y': 48, 'x': 39}, {'y': 65, 'x': 26}]",2565548,Pass,682,1H,166.859710,...,0,0,0,1,0,0,0,0,0,1
43,8,High pass,"[{'id': 901}, {'id': 1802}]",3542,"[{'y': 67, 'x': 32}, {'y': 42, 'x': 74}]",2565548,Pass,682,1H,181.665639,...,0,0,0,0,1,0,0,0,0,1
50,8,Simple pass,[{'id': 1801}],3542,"[{'y': 60, 'x': 35}, {'y': 97, 'x': 42}]",2565548,Pass,682,1H,213.736626,...,0,0,0,0,1,0,0,0,0,1
60,8,Head pass,[{'id': 1801}],3542,"[{'y': 25, 'x': 26}, {'y': 9, 'x': 27}]",2565548,Pass,682,1H,278.645906,...,0,0,1,0,0,0,0,0,0,1
85,8,Smart pass,"[{'id': 901}, {'id': 1802}]",3542,"[{'y': 61, 'x': 38}, {'y': 47, 'x': 54}]",2565548,Pass,682,1H,354.921472,...,0,0,0,0,1,0,0,0,0,1
105,8,Simple pass,[{'id': 1801}],3542,"[{'y': 71, 'x': 32}, {'y': 96, 'x': 42}]",2565548,Pass,682,1H,440.315069,...,0,0,0,0,1,0,0,0,0,1
199,8,Simple pass,[{'id': 1801}],3542,"[{'y': 77, 'x': 45}, {'y': 72, 'x': 32}]",2565548,Pass,682,1H,844.316154,...,0,0,0,0,0,1,0,0,0,1
205,8,Simple pass,[{'id': 1801}],3542,"[{'y': 87, 'x': 42}, {'y': 38, 'x': 38}]",2565548,Pass,682,1H,862.919269,...,0,0,0,0,0,0,1,0,0,1
240,8,Simple pass,[{'id': 1801}],3542,"[{'y': 23, 'x': 30}, {'y': 19, 'x': 30}]",2565548,Pass,682,1H,1004.295944,...,0,0,1,0,0,0,0,0,0,1


## Export 

In [44]:
fields = ['gameweek','matchId', 'teamId', 'name', 'playerId', 'shortName', 'eventName', 'subEventName', 'matchPeriod', 'eventSec' , 'st_x', 'st_y', 'ed_x', 'ed_y', 'success'] 
fields +=list(map(lambda v : 'v_'+ v[0], vertical_zones)) 
fields +=list(map(lambda v : 'h_'+ v[0], horizontal_zones))
fields +=['won','drawn', 'lost']

fields

['gameweek',
 'matchId',
 'teamId',
 'name',
 'playerId',
 'shortName',
 'eventName',
 'subEventName',
 'matchPeriod',
 'eventSec',
 'st_x',
 'st_y',
 'ed_x',
 'ed_y',
 'success',
 'v_left',
 'v_middle',
 'v_right',
 'h_own_low',
 'h_own_middle',
 'h_own_high',
 'h_rival_low',
 'h_rival_middle',
 'h_rival_high',
 'won',
 'drawn',
 'lost']

In [49]:
df_save = df_pass[fields].sort_values(by=['matchId','eventSec'])

In [46]:
df_save_match = df_pass[fields].loc[df_pass.matchId==2565548].sort_values(by=['eventSec'])

## Save file

In [50]:
df_save.to_csv(os.path.join(processed_dir, 'passing.csv'), index=False)

In [48]:
df_save_match.to_csv(os.path.join(processed_dir, 'passing_match.csv'), index=False)